In [3]:
import pandas as pd
import numpy as np
import pickle
from prophet import Prophet
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error


ruta_datos = "/kaggle/input/datos-procesados/DF_DEMANDA_10_25_PROCESADO.csv"

df = pd.read_csv(ruta_datos, parse_dates=["fecha"])
df = df.rename(columns={"fecha": "ds", "valor_(GWh)": "y"})

min_val = df["y"].min()
max_val = df["y"].max()

#  Agregar la variable exógena
df["es_festivo"] = df["es_festivo"] 

#  Entrenar Prophet **CON `es_festivo`**
modelo_con_exogena = Prophet()
modelo_con_exogena.add_regressor("es_festivo")
modelo_con_exogena.fit(df)

#  Entrenar Prophet **SIN `es_festivo`**
modelo_sin_exogena = Prophet()
modelo_sin_exogena.fit(df.drop(columns=["es_festivo"]))  # Quitamos la variable exógena

#  Guardar modelos en Kaggle
ruta_modelo_con = "/kaggle/working/modelo_prophet_con.pkl"
ruta_modelo_sin = "/kaggle/working/modelo_prophet_sin.pkl"

with open(ruta_modelo_con, "wb") as f:
    pickle.dump({"modelo": modelo_con_exogena, "min_val": min_val, "max_val": max_val}, f)

with open(ruta_modelo_sin, "wb") as f:
    pickle.dump({"modelo": modelo_sin_exogena, "min_val": min_val, "max_val": max_val}, f)

print(f"Modelos guardados en: {ruta_modelo_con} y {ruta_modelo_sin}")

#  Generar predicciones para ambas versiones
df_futuro = df[["ds", "es_festivo"]].copy()

#  Predicción **CON** es_festivo
pred_con = modelo_con_exogena.predict(df_futuro)[["ds", "yhat"]]

#  Predicción **SIN** es_festivo
pred_sin = modelo_sin_exogena.predict(df_futuro[["ds"]])[["ds", "yhat"]]

# Desescalar valores correctamente 
pred_con["yhat"] = pred_con["yhat"] * (max_val - min_val) + min_val
pred_sin["yhat"] = pred_sin["yhat"] * (max_val - min_val) + min_val

# Juntar predicciones y valores reales
df_resultado = df.merge(pred_con, on="ds", how="inner", suffixes=("_real", "_con"))
df_resultado = df_resultado.merge(pred_sin, on="ds", how="inner", suffixes=("", "_sin"))

#  Calcular métricas para ambas versiones
metricas = {}

for version in ["", "sin"]:  #  "yhat" y "yhat_sin" son las predicciones
    r2 = r2_score(df_resultado["y"], df_resultado[f"yhat{('_' + version) if version else ''}"])
    mae = mean_absolute_error(df_resultado["y"], df_resultado[f"yhat{('_' + version) if version else ''}"])
    rmse = np.sqrt(mean_squared_error(df_resultado["y"], df_resultado[f"yhat{('_' + version) if version else ''}"]))

    metricas[version if version else "con"] = {"r2": r2, "mae_GWh": mae, "rmse_GWh": rmse}  #  Renombramos la clave "con"

print("Comparación de métricas entre con es_festivo y sin es_festivo:")
for version, valores in metricas.items():
    print(f"\n **Métricas {version} es_festivo**:")
    for key, value in valores.items():
        print(f"{key}: {value:.5f}")


df_metricas = pd.DataFrame.from_dict(metricas, orient="index")
ruta_csv = "/kaggle/working/predicciones_comparacion_prophet.csv"
df_metricas.to_csv(ruta_csv)

print(f"Métricas guardadas en: {ruta_csv}")

15:51:50 - cmdstanpy - INFO - Chain [1] start processing
15:51:51 - cmdstanpy - INFO - Chain [1] done processing
15:51:51 - cmdstanpy - INFO - Chain [1] start processing
15:51:52 - cmdstanpy - INFO - Chain [1] done processing


Modelos guardados en: /kaggle/working/modelo_prophet_con.pkl y /kaggle/working/modelo_prophet_sin.pkl
Comparación de métricas entre con es_festivo y sin es_festivo:

 **Métricas con es_festivo**:
r2: 0.82940
mae_GWh: 0.05122
rmse_GWh: 0.06822

 **Métricas sin es_festivo**:
r2: 0.77271
mae_GWh: 0.05672
rmse_GWh: 0.07874
Métricas guardadas en: /kaggle/working/predicciones_comparacion_prophet.csv
